# API helper calls

Try to GET data from a url. If no response, retry `n_tries` times with exponential backoff.

In [109]:
possibly_get_data <- function(url, n_tries, ...){

    rate <- rate_backoff(pause_base = 1, max_times = n_tries)
    possibly_insistent_get <- insistently(GET, rate, quiet = FALSE) %>% possibly(otherwise = NULL)

    possibly_insistent_get(url, ...)
}

In [1]:
testResponse <- function (response_object) {
    if(is.null(response_object)) {
        print("\nUnable to fetch updated data, is the network up?")
    }
    else if(response_object$status_code < 200 | response_object$status_code >= 400) {
        print(paste("\nUnable to update data \nstatus code:", x$status_code, "\nerror messge:", x$error))
    }
    else {
        print("Successfully fetched data")
    }
}

# Get base station parameters

Check to see if a station_elevation is present, and if NULL, use the geonames.org webservice to provide an elevation for a given latitude and longitude

In [110]:
findStationElevation <- function(station_elevation, station_latitude, station_longitude, geonames_username) {
    if (is.null(station_elevation)) { 
        station_elevation <- possibly_get_data(
            paste0("http://api.geonames.org/gtopo30JSON?lat=", station_latitude, 
                   "&lng=", station_longitude, "&username=", 
                   geonames_username), 8) %>% content %>% pluck("gtopo30")}
    station_elevation
}

Perform a sanity check that the given coordinates are near where you expect them

In [ ]:
findNearbyCity <- function(station_latitude, station_longitude, geonames_username) {
    possibly_get_data(paste0("http://api.geonames.org/findNearbyPlaceNameJSON?lat=", 
                                              station_latitude, "&lng=", station_longitude, "&username=", 
                                              geonames_username), 8) %>% content
}

# Get dongle parameters

## Parse the output of rtl_test to find the maximum gain of the dongle

Need to work on suppressing warnings

In [100]:
findDongleMaxGain <- function(gain) {
    if (is.null(gain)) {
        rtl_test_output <- system2('timeout', c('2', 'rtl_test'), stdout = TRUE, stderr = TRUE)
        if(any(grepl("Failed to open", rtl_test_output))) {
            print("Unable to open RTL SDR device. Is it busy or missing?")
            stop()
        }
        if(any(grepl("Supported gain", rtl_test_output))) {
            gain <- rtl_test_output %>% pluck(grep("Supported", .)) %>% strsplit(" ") %>% 
                unlist %>% tail(n=1) %>% as.numeric
        }
    }
    return(gain)
}

## Find the offset for the internal clock on the dongle. 
Going to use the `kal` command and for this we need to set the gain, so first check to see if that's in the config.  Then use `kal` to find a nearby GSM station in the GSM900 band with a default offset of 0. Then use that station to test the offset

In [99]:
findClockOffset <- function(clock_offset, gain) {
    if (is.null(gain)) {
        gain <- findDongleMaxGain(gain)
    }
    if (is.null(clock_offset)) {
        print("No clock offset found. Running kal. This will take a few minutes")
        flush.console()
        best_channel <- system(paste0("kal -s GSM900 -e 0 -g ", gain, " | sed 's/ \\+/\\t/g' | cut -f 3,8 | ",
                              "sort -k2 -n -r | head -1 | cut -f 1"),
                       intern = TRUE)
        if(length(best_channel > 0)) {
            print("GSM station found. Calibrating dongle drift")
            flush.console()
            clock_offset <- system(paste0("kal -c ", best_channel, " -g ", gain, " -e 0"), intern=TRUE) %>%
                pluck(5) %>% strsplit(":") %>% pluck(1,2) %>% substr(2,nchar(.) - 4) %>% as.numeric
            }
        else {
            print("error finding gsm channel")
            clock_offset <- NULL
        }
    }
    return(clock_offset)
}

# Access the N2YO API

In [2]:
generateN2YOURL <- function(config_list) {
    config_list$satellites %>% 
    map("norad_id") %>% 
    paste("https://www.n2yo.com/rest/v1/satellite/radiopasses", ., 
          config_list$station_latitude, config_list$station_longitude, config_list$station_elevation, 
          10, config_list$minimum_observable_elevation, config_list$n2yo_api_key, sep="/")}

# Scheduling recordings

Looks for pairs of rows that have overlapping times, then selects the row with the higher priority value (since a low priority number eg 1 is what we want to record). 

May break if there are three consecutive passes that overlap...

In [3]:
passesToDrop <- function(df) {
    df %>% 
        filter(endUTC > lead(startUTC) | lag(endUTC) > startUTC) %>% 
        mutate(group = ({seq(1, nrow(.)/2)} %>% 
        rep(each=2))) %>% 
        group_by(group) %>% 
        filter(priority == max(priority))
    }